In [1]:
# Import modules
import os
import numpy as np
import pandas as pd
import xnat
import pydicom
import matplotlib.pyplot as plt
import re

In [2]:
# Connect to your XNAT Session URL and XNAT User ID
session = xnat.connect('http://localhost', user = 'admin')

Please enter the password for user 'admin': ········


In [3]:
rsna_project = session.projects["20231206"]
lidc_project = session.projects["20231211"]

## Searching
- https://xnat.readthedocs.io/_/downloads/en/0.3.26/pdf/

In [11]:
dir(session.classes)

['AbstractDemographicData',
 'AbstractExecutionEnvironment',
 'AbstractProjectAsset',
 'AbstractProtocol',
 'AbstractResource',
 'AbstractResourceTags',
 'AbstractStatistics',
 'AbstractSubjectMetadata',
 'AccessLog',
 'ActionType',
 'AddField',
 'AddFieldString',
 'AddIDString',
 'AdditionalStatisticsDouble',
 'AdditionalVal',
 'Algorithm',
 'AliasString',
 'BuildData',
 'BytesIO',
 'Catalog',
 'CatalogMetafields',
 'ChangeInfo',
 'CommentString',
 'CompilerString',
 'ComputationData',
 'ContentString',
 'ContrastBolus',
 'CrScanData',
 'CrSessionData',
 'Criteria',
 'CriteriaSet',
 'CsvValuesString',
 'CtQcScanData',
 'CtScanData',
 'CtScanDataDcmvalidation',
 'CtScanDataParameters',
 'CtScanDataParametersCollimationwidth',
 'CtScanDataParametersDerivation',
 'CtScanDataParametersEstimateddosesaving',
 'CtScanDataParametersFov',
 'CtScanDataParametersRescale',
 'CtScanDataParametersVoxelres',
 'CtSessionData',
 'DatatypeProtocol',
 'DcmCatalog',
 'DcmCatalogDimensions',
 'DcmCatalogV

### Accessing DICOM headers of scan

`session.service.dicom_dump(scan_uri)` 사용하면 되는데, scan_uri는 어떻게 한번에 뽑지? 
- `ScanData.dicom_dump(fields=None)` 에서, fields to filter for DICOM tags, it can either a tag name or tag number. returns JSON object (dict) representation of header

#### pydicom
- A limitation of the dicom_dump of XNAT is that field values are truncated under 64 characters. 
- If you want to access 
the entire dicom header, a convenience method is added that reads the header vi `pydicom`

In [64]:
def scanframe(project_name, subject_name, experiment_name, scan_name): 
    # ID or Accession number
    scan = session.projects[project_name].subjects[subject_name].experiments[experiment_name].scans[scan_name]
    header = scan.read_dicom()
    
    df = pd.DataFrame(header.values())

    df[0] = df[0].apply(lambda x: pydicom.dataelem.DataElement_from_raw(x) if isinstance(x, pydicom.dataelem.RawDataElement) else x)
    df['tag'] = df[0].apply(lambda x: pydicom.tag.Tag(x) if isinstance(x, pydicom.tag.BaseTag) else x)
    df['tag'] = df['tag'].astype('str')
    pattern = r'\(\w{4},\s*\w{4}\)'  
    df['tag'] = df['tag'].apply(lambda x: re.search(pattern, x).group() if re.search(pattern, x) else None)
    
    df['name'] = df[0].apply(lambda x: x.name)
    df['value'] = df[0].apply(lambda x: x.value)
    
    df = df[['tag', 'name', 'value']]

    return df

In [65]:
# example
scanframe('20231206', '00337', '00337', 'T2w')

,tag,name,value
0,"(0008, 0005)",Specific Character Set,ISO_IR 100
1,"(0008, 0008)",Image Type,"[DERIVED, SECONDARY]"
2,"(0008, 0016)",SOP Class UID,1.2.840.10008.5.1.4.1.1.4
3,"(0008, 0018)",SOP Instance UID,1.2.826.0.1.3680043.8.498.42164575998861237205...
4,"(0008, 0050)",Accession Number,00337
5,"(0008, 0060)",Modality,MR
6,"(0008, 103e)",Series Description,T2w
7,"(0010, 0010)",Patient's Name,"(0, 0, 3, 3, 7)"
8,"(0010, 0020)",Patient ID,00337
9,"(0018, 0023)",MR Acquisition Type,2D


### Extract the ID or Accession # of Projects/ Subjects/ Experiments/ Scans